In [2]:
import pandas as pd
import datetime
photoMealsDf = pd.read_excel(io="../data/ODS/ADMU Extract - 11420.ods", sheet_name="Food Photo - Portions", engine="odf")
fnddsDf = pd.read_excel(io="../data/ODS/ADMU Extract - 11420.ods", sheet_name="FnddsGlance", engine="odf")

In [3]:
filt1 = photoMealsDf['Meal'].isin(["LabMeal", "TrainingMeal"])
anaPhotoMealsDf = photoMealsDf[~filt1]
participantInfoDf = anaPhotoMealsDf[anaPhotoMealsDf.columns[:19]].drop(columns=['PercentEaten'])

PercentEaten = anaPhotoMealsDf['PercentEaten'].transpose()

photoMealsDfColumnsToAnalyzeDf = anaPhotoMealsDf[anaPhotoMealsDf.columns[19:]]
photoMealsDfResult = photoMealsDfColumnsToAnalyzeDf.mul(PercentEaten, axis=0)

anaPhotoMealsDf = pd.concat([participantInfoDf, photoMealsDfResult], axis=1)

In [4]:
filt2 = fnddsDf['Food code'].isin(anaPhotoMealsDf['FoodCode'])

foodsConsumedDf = fnddsDf[filt2].drop_duplicates()
foodsConsumedDf = foodsConsumedDf.set_index(['Food code'])

In [5]:
hundredGramsDf = foodsConsumedDf.loc[anaPhotoMealsDf['FoodCode']]
anaHundredGramsDf = hundredGramsDf[hundredGramsDf.columns[3:]]
oneGramDf = anaPhotoMealsDf['TemplateGramWt'].div(100)
fnddsVals = anaHundredGramsDf.mul(oneGramDf.values, axis=0)
final = pd.concat([anaPhotoMealsDf.reset_index(), fnddsVals.reset_index()], axis=1)

In [6]:
final.head()

,index,SubjectID,Date,Time,DateAsString,Meal,FoodPhotoEventID,Food,FoodDictionaryIndex,FoodCode,...,20:1 (g),22:1 (g),18:2 (g),18:3 (g),18:4 (g),20:4 (g),20:5 n-3 (g),22:5 n-3 (g),22:6 n-3 (g),Water (g)
0,16,A00-07-1409,2020-09-06,09:37:03,09-06-2020_09:37:03,Breakfast,8F53EDEE-1C9D-43A6-B0B9-619148E47A7F,Cream,17,14301010,...,0.025984,0.000812,0.492072,0.053186,0.000406,0.021518,0.004060,0.008120,0.000000,21.363720
1,17,A00-07-1409,2020-09-06,09:37:03,09-06-2020_09:37:03,Breakfast,8F53EDEE-1C9D-43A6-B0B9-619148E47A7F,White Bagel,576,51180010,...,0.002977,0.000000,0.299461,0.014288,0.000000,0.000000,0.000000,0.000000,0.000000,20.128783
2,18,A00-07-1409,2020-09-06,12:47:24,09-06-2020_12:47:24,Lunch,286BC92F-3189-41D9-92AB-6ED06CB94479,Crackers Regular,649,54001000,...,0.047700,0.004500,5.206950,0.695700,0.000000,0.004500,0.000000,0.000000,0.000000,1.413000
3,19,A00-07-1409,2020-09-06,12:47:24,09-06-2020_12:47:24,Lunch,286BC92F-3189-41D9-92AB-6ED06CB94479,Ham and Cheese Sandwich,362,27520320,...,0.095681,0.002126,4.388580,0.559204,0.002126,0.046777,0.004253,0.004253,0.002126,111.840750
4,20,A00-07-1409,2020-09-06,12:47:24,09-06-2020_12:47:24,Lunch,286BC92F-3189-41D9-92AB-6ED06CB94479,Hummus,792,41208030,...,0.001800,0.001800,0.080280,0.064800,0.000000,0.000000,0.000000,0.000000,0.000000,25.732801


In [ ]:
date = datetime.date.today()
filename = f'FoodPhotoData-{date}.xlsx'
final.to_excel(filename)